# Algoritmo para buscar el archivo y cargarlo en la base de datos 

## Configuracion inicial

In [1]:
import mysql.connector
import pandas as pd
import glob
import os
import datetime
from datetime import datetime

# Conexión a la base de datos
db = mysql.connector.connect(
    host="192.168.1.66",
    user="root",
    password="admin",
    database="Stock"
)

cursor = db.cursor(buffered=True)

## Algoritmo para buscar el archivo y hacer las transformaciones

Obtenemos la ultima fecha cargada en el df

In [2]:
query = "SELECT Fecha from Roturas"
df = pd.read_sql(query, db)
fecha_maxima = df['Fecha'].max()

C:\Users\mdasilva\AppData\Local\Temp\ipykernel_12808\755416127.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, db)


Buscamos los archivos que tengan una fecha de modificacion mayor a la ultima fecha cargada en la base de datos

In [4]:
ruta_base = r"\\layla\\Documentos\\STOCK\\"
ruta_carpetas_base = glob.glob(ruta_base + 'Sto*')
rutas_imagen = []

for ruta in ruta_carpetas_base:
    # Construye la ruta hacia la carpeta "10- ROTURAS"
    ruta_roturas = os.path.join(ruta, '10- ROTURAS')
    
    # Verifica si la carpeta "10- ROTURAS" existe
    if os.path.exists(ruta_roturas):
        # Construye la ruta hacia la carpeta "IMAGEN" dentro de "10- ROTURAS"
        ruta_imagen = os.path.join(ruta_roturas, 'IMAGEN')

        # Verifica si la carpeta "IMAGEN" existe
        if os.path.exists(ruta_imagen):
            rutas_imagen.append(ruta_imagen)

rutas_meses = []

for ruta_imagen in rutas_imagen:
    # Busca todas las subcarpetas dentro de la carpeta "IMAGEN"
    subcarpetas = glob.glob(os.path.join(ruta_imagen, '*/'))

    # Añade las rutas de las subcarpetas a la lista rutas_meses
    rutas_meses.extend(subcarpetas)

rutas_archivos_recientes = []

for ruta_mes in rutas_meses:
    # Lista todos los archivos en la subcarpeta
    archivos = glob.glob(os.path.join(ruta_mes, '*'))

    for archivo in archivos:
        # Obtiene la fecha de modificación del archivo
        fecha_modificacion_datetime = datetime.fromtimestamp(os.path.getmtime(archivo))
        fecha_modificacion = fecha_modificacion_datetime.date()

        # Compara la fecha de modificación con la fecha max
        if fecha_modificacion > fecha_maxima:
            rutas_archivos_recientes.append(archivo)

Con los archivos ya ubicados, procedemos a cargarlos en un df y hacer las modificaciones necesarias

In [6]:
datos = []
for x in rutas_archivos_recientes:
    y = pd.read_excel(x)
    datos.append(y)

datos = pd.concat(datos)

datos = datos[['fecha', 'articulo', 'cantidad', 'costo', 'nombre']]

datos['fecha'] = datos['fecha'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y').date() if pd.notnull(x) else x)

datos['articulo'] = datos['articulo'].astype(str)

datos.dropna(inplace= True)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


## Cargar los datos en la BD

In [70]:
# Abre un archivo para registrar los errores
with open("errores_log.txt", "w") as error_log:

    for index, fila in datos.iterrows():
        try:
            # Preparar tu consulta SQL y los valores
            query = "INSERT INTO Roturas (Fecha, Codigo, Cantidad, Costo, Motivo) VALUES (%s, %s, %s, %s, %s)"
            values = (fila['fecha'], fila['articulo'], fila['cantidad'], fila['costo'], fila['nombre'])

            # Ejecutar la consulta
            cursor.execute(query, values)
            db.commit()

        except mysql.connector.IntegrityError as e:
            print(f"Error al insertar la fila {index}: {e}")
            
            # Escribir el error y el código problemático en el archivo de log
            error_log.write(f"Fila {index}, Código: {fila['articulo']}, Error: {e}\n")
            
            # Continuar con la siguiente fila
            continue


Error al insertar la fila 0: 1062 (23000): Duplicate entry '322-2023-12-14-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 1: 1062 (23000): Duplicate entry '350-2023-12-14-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 2: 1062 (23000): Duplicate entry '350-2023-12-14-ROTURA DEPOSITO' for key 'Roturas.PRIMARY'
Error al insertar la fila 3: 1062 (23000): Duplicate entry '353-2023-12-14-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 4: 1062 (23000): Duplicate entry '1657-2023-12-14-VENCIMIENTO' for key 'Roturas.PRIMARY'
Error al insertar la fila 5: 1062 (23000): Duplicate entry '1689-2023-12-14-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 6: 1062 (23000): Duplicate entry '3340-2023-12-14-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 7: 1062 (23000): Duplicate entry '3342-2023-12-14-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 8: 1062 (23000): Duplicate entry '3751-2023-12-14-ROTURA' for key 'Roturas.PRIMARY'
